In [1]:
import numpy as N

# trial for hg

def ApproximateJacobian(f, x, dx=1e-6):
    """Return an approximation of the Jacobian Df(x) as a numpy matrix"""
    try:
        n = len(x)
    except TypeError:
        n = 1 
    fx = f(x)
    Df_x = N.matrix(N.zeros((n,n)))
    for i in range(n):
        v = N.matrix(N.zeros((n,1)))
        v[i,0] = dx
        Df_x[:,i] = (f(x + v) - fx)/dx # BUG FIX: divide by dx
    return Df_x


class Polynomial(object):
    """Callable polynomial object.

    Example usage: to construct the polynomial p(x) = x^2 + 2x + 3,
    and evaluate p(5):

    p = Polynomial([1, 2, 3])
    p(5)"""

    def __init__(self, coeffs):
        self._coeffs = coeffs

    def __repr__(self):
        return "Polynomial(%s)" % (", ".join([str(x) for x in self._coeffs]))

    def f(self,x):
        ans = self._coeffs[0]
        for c in self._coeffs[1:]:
            ans = x*ans + c
        return ans

    def __call__(self, x):
        return self.f(x)

# extra nonlinear functions and their analytical derivatives for testing

def MyGaussian(x):  # gaussian
    return N.exp(-x**2 /2)

def MyGaussian_der(x): # derivative of the gaussian
    return -x * N.exp(-x**2 /2) 
 
def MyNonLinear2D_1(x): 
    x1 = N.exp(x[0,0]) + N.sin(N.arcsin(1) * x[1,0])+  x[0,0]*x[1,0] -2
    x2 = 1/(1+x[0,0]**2) + N.log(x[1,0]) +N.exp(x[0,0]+x[1,0]) - 1 - N.exp(1) 
    return N.matrix([[x1],[x2]]) 

def MyNonLinear2D_1_der(x): 
    y11 = N.exp(x[0,0])+  x[1,0] 
    y12 = N.arcsin(1)* N.cos(N.arcsin(1) * x[1,0])+  x[0,0] 
    y21 = -2*x[0,0]/(1+x[0,0]**2)**2 + N.exp(x[0,0]+x[1,0]) 
    y22 = 1 / x[1,0] +N.exp(x[0,0]+x[1,0]) 
    return N.matrix([[y11, y12],[y21 , y22]]) 


In [2]:
#!/usr/bin/env python

import functions as F
import numpy as N
import unittest

class TestFunctions(unittest.TestCase):
    def testApproxJacobian1(self):
        slope = 3.0
        def f(x):
            return slope * x + 5.0
        x0 = 2.0
        dx = 1.e-3
        Df_x = F.ApproximateJacobian(f, x0, dx)
        self.assertEqual(Df_x.shape, (1,1))
        self.assertAlmostEqual(Df_x, slope)

    def testApproxJacobian2(self):
        A = N.matrix("1. 2.; 3. 4.")
        def f(x):
            return A * x
        x0 = N.matrix("5; 6")
        dx = 1.e-6
        Df_x = F.ApproximateJacobian(f, x0, dx)
        self.assertEqual(Df_x.shape, (2,2))
        N.testing.assert_array_almost_equal(Df_x, A) 

    # used less special coordinates to make sure it works, changed
    # asserEqual to assertAlmostEqual to handle double precision
    def testPolynomial(self):
        # p(x) = x^2 + 2x + 3
        p = F.Polynomial([1.3, 2.7, 8.6])
        for x in N.linspace(-2,2,11):
            self.assertAlmostEqual(p(x), 1.3*x**2 + 2.7*x + 8.6)

    # NOVEL TESTS

    def testAnalyticalNumericalCompare_1D(self): 
        # compares analytical and numerical Jacobians for scalars
        x0 = 1.34;
        dx = 1e-10;
        Df_x = F.ApproximateJacobian(F.MyGaussian, x0, dx)
        DfA = F.MyGaussian_der(x0)
        self.assertAlmostEqual(Df_x, DfA) 

    def testAnalyticalNumericalCompare_2D(self): 
        # compares analytical and numerical Jacobians for 2D
        x0 = N.matrix([[1.23],[1.2343]])
        #dx chosen so that dx**2 (rough numerical error) < double precision
        dx = 1e-8;
        Df_x = F.ApproximateJacobian(F.MyNonLinear2D_1, x0, dx)
        DfA = F.MyNonLinear2D_1_der(x0)
        N.testing.assert_array_almost_equal(N.array(Df_x), N.array(DfA)) 


if __name__ == '__main__':
    unittest.main()


E
ERROR: /Users/fethimubin/Library/Jupyter/runtime/kernel-4de55be8-d53c-4285-866e-a9f21c769f02 (unittest.loader._FailedTest)
----------------------------------------------------------------------
AttributeError: module '__main__' has no attribute '/Users/fethimubin/Library/Jupyter/runtime/kernel-4de55be8-d53c-4285-866e-a9f21c769f02'

----------------------------------------------------------------------
Ran 1 test in 0.001s

FAILED (errors=1)


SystemExit: True

/Users/Shared/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3334: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
